In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# 0. Supplementary Knowledge
먼저, 밑에 것 보기 전에 미리 공부해야할 것들을 보자

# 1. Basic Setup

## 1.1. Importing files

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy # Add Deepcopy for args
import seaborn as sns
import matplotlib.pyplot as plt
import nibabel as nib
import glob
import os
from tqdm.auto import tqdm
from nilearn import plotting
import matplotlib.pyplot as plt
import pandas as pd
import random
import hashlib
import json
from os import listdir
from os.path import isfile, join
import monai
from monai.data import CSVSaver, ImageDataset, DistributedWeightedRandomSampler
from monai.transforms import AddChannel, Compose, RandRotate90, Resize, ScaleIntensity, Flip, ToTensor
from monai.utils import set_determinism
from monai.apps import CrossValidation

/home/connectome/dyhan316/.conda/envs/VAE/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1.2. defining args/device
원래는 parser args로 하지만, 여기서는 notebook이니 그냥 args만들어서 하자

**must be filled (set as `None` for now)**
* `model`
* `optim` (Adam으로 지정해 주기는 함)
* `exp_name` (뭔지 몰라서 지정 딱히 안해줌)

In [5]:
class Args():
    def __init__(self):
        self.GPU_NUM =1
        self.model = 'CNN3D_1' ##일단은 이것을 돌려보자 (CNN3D_2 도 있기는 함!) 
        self.val_size=0.1
        self.test_size=0.1
        self.resize=(96,96,96)
        self.train_batch_size=16
        self.val_batch_size=8
        self.test_batch_size =8
        self.in_channels =1
        self.out_dim=2
        self.optim = "Adam" #그냥 임의로 내가 넣은 것
        self.lr=1e-5
        self.l2=0.00001
        self.epoch = 10 #그냥 임의로 내가 정한 것 
        self.exp_name = "test_test" #experiment name인것 같다..?
args = Args()
##잘됬는지 중간 점검
print(args.epoch)

10


In [9]:
## ========= GPU Setting ========= ##
GPU_NUM = args.GPU_NUM # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device)
print('Experiment will be performed on GPU {}'.format(torch.cuda.current_device()))

Experiment will be performed on GPU 1


## X.X. dsfkjdklsfjdklsfjdklsfds

**밑에 고치기!! 일단은 skip (어떤 것인지 등등도 해야하나 skip)**

In [11]:
## ========= Data Preprocessing ========= ##
# getting image file names (subject ID + '.npy') as list
base_dir = '/scratch/connectome/dhkdgmlghks/docker/share/preprocessed_masked'
data_dir = '/scratch/connectome/dhkdgmlghks/docker/share/preprocessed_masked'
os.chdir(data_dir)
image_files = glob.glob('*.npy')
image_files = sorted(image_files)

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/connectome/dhkdgmlghks/docker/share/preprocessed_masked'

In [14]:

# getting subject ID and target variables
target = 'sex'

subject_data = pd.read_csv('/scratch/connectome/3DCNN/data/1.ABCD/4.demo_qc/ABCD_phenotype_total.csv')
subject_data = subject_data.loc[:,['subjectkey',target]]
subject_data = subject_data.sort_values(by='subjectkey')
subject_data = subject_data.dropna(axis = 0)
subject_data = subject_data.reset_index(drop=True) # removing subject have NA values in sex

subject_data


            subjectkey    age  sex  race.ethnicity  high.educ  income  \
0      NDARINV003RTV85  131.0  2.0             1.0       13.0     8.0   
1      NDARINV005V6D2C    NaN  NaN             NaN        NaN     NaN   
2      NDARINV007W6H7B  126.0  1.0             1.0       19.0    10.0   
3      NDARINV00BD7VDC  112.0  1.0             1.0       20.0    10.0   
4      NDARINV00CY2MDM  130.0  1.0             1.0       15.0     6.0   
...                ...    ...  ...             ...        ...     ...   
11870  NDARINVZZNX6W2P  131.0  1.0             1.0       18.0     9.0   
11871  NDARINVZZPKBDAC  113.0  2.0             1.0       19.0    10.0   
11872  NDARINVZZZ2ALR6  120.0  2.0             5.0       21.0    10.0   
11873  NDARINVZZZNB0XC  108.0  2.0             3.0       19.0     3.0   
11874  NDARINVZZZP87KR  126.0  2.0             2.0       17.0     7.0   

       married  abcd_site  height  weight  ...  Substance.Use.Disorder.x  \
0          1.0        6.0  56.500    93.0  ... 

,subjectkey,sex
0,NDARINV003RTV85,2.0
1,NDARINV007W6H7B,1.0
2,NDARINV00BD7VDC,1.0
3,NDARINV00CY2MDM,1.0
4,NDARINV00HEV6HB,1.0
...,...,...
11526,NDARINVZZNX6W2P,1.0
11527,NDARINVZZPKBDAC,2.0
11528,NDARINVZZZ2ALR6,2.0
11529,NDARINVZZZNB0XC,2.0
